<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myXH_Return_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import yfinance as yf
import pandas as pd

def calculate_stock_gains(start_date, end_date, stock_list):
    """
    Calculate percentage gains for a list of NSE stocks over a specified date range.

    Args:
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
        stock_list (list): List of NSE stock symbols (e.g., ['RELIANCE.NS', 'TCS.NS']).

    Returns:
        pd.DataFrame: DataFrame containing stock symbols and their percentage gains.
    """
    data = []

    for stock in stock_list:
        try:
            # Fetch stock data for the given date range
            stock_data = yf.Ticker(stock + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            stock_data = stock_data[(stock_data.index <= end_date) & (stock_data.index >= start_date)]

            if not stock_data.empty:
                # Calculate percentage gain
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percentage_gain = ((end_price - start_price) / start_price) * 100

                data.append({
                    'Stock': stock,
                    'Start Price': start_price,
                    'End Price': end_price,
                    'Gains (%)': round(percentage_gain, 2)
                })
            else:
                data.append({
                    'Stock': stock,
                    'Start Price': None,
                    'End Price': None,
                    'Gains (%)': None
                })
        except Exception as e:
            data.append({
                'Stock': stock,
                'Start Price': None,
                'End Price': None,
                'Gains (%)': None,
                'Error': str(e)
            })

    # Create and return DataFrame
    return pd.DataFrame(data)

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# Example usage:
if __name__ == "__main__":
    start_date = "2024-09-27"
    end_date = "2025-04-22"


    pps_df = get_mypps_df()
    scr_df = get_myscreen_df()

    df = pd.merge(pps_df, scr_df, on = 'Symbol')

    #stock_list = df[(df['Conviction'].isin(['X','H'])) & ((df['CapType'] == 'LC'))]['Symbol'].values

    list_x40 = list(pps_df[pps_df['MBQ'].str.contains('X40', na=False)]['Symbol'].values)
    list_x5k = list(pps_df[pps_df['MBQ'].str.contains('X5K', na=False)]['Symbol'].values)
    list_x40n = list(pps_df[pps_df['MBQ'].str.contains('X40N', na=False)]['Symbol'].values)
    #len(set(list_x40 + list_x5k) - set( list_x40n))
    stock_list = list(set(list_x40 + list_x5k) - set( list_x40n))

    #stock_list = ["RELIANCE", "TCS", "INFY", "HDFCBANK"]

    result_df = calculate_stock_gains(start_date, end_date, stock_list)
    print(str(round(result_df['Gains (%)'].mean())) + '%')

result_df.sort_values(by = 'Gains (%)', ascending=False)

-9%


,Stock,Start Price,End Price,Gains (%)
20,KOTAKBANK,1872.449951,2268.800049,21.17
51,BAJFINANCE,7756.000000,9260.500000,19.40
4,BAJAJHLDNG,10543.450195,12324.000000,16.89
33,HDFCBANK,1752.650024,1961.699951,11.93
19,INDIGO,4945.700195,5534.000000,11.90
...,...,...,...,...
64,COLPAL,3734.585938,2658.800049,-28.81
32,BAJAJ-AUTO,12666.400391,8141.000000,-35.73
68,TATAMOTORS,993.000000,630.849976,-36.47
59,INDUSINDBK,1462.699951,787.500000,-46.16
